### **Intro**
We're going to start off by doing some data preprocessing for an emotion classifier model that takes a piece of audio data and labels it.

In [ ]:
# Libraries to install
!python -m pip install librosa
# !python -m pip install matplotlib==3.3.4
# !python -m pip install datasets[audio]

In [27]:
import os
import librosa
import matplotlib.pyplot as plt
import librosa.display as display
import pandas as pd

# Let's analyze some of our audio files
audio_folder_name = "data\CREMA"
audio_files = os.listdir(audio_folder_name)

for file_name in audio_files[:5]:
    file_path = os.path.join(audio_folder_name, file_name)
    y, sr = librosa.load(file_path, sr=None)

    # Info about sampling rate
    print('Length of the rAV audio audio: {:.2f} seconds'.format(len(y) / sr))
    print('Number of samples: {}'.format(len(y)))
    print('Sampling rate: {} Hz'.format(sr))

    # About mfccs
    # mfccs = librosa.feature.mfcc(y=array, sr=sampling_rate)
    # print(mfccs)
    # df = pd.DataFrame(mfccs)
    # print(df.head())

Length of the rAV audio audio: 2.28 seconds
Number of samples: 36409
Sampling rate: 16000 Hz
Length of the rAV audio audio: 2.34 seconds
Number of samples: 37371
Sampling rate: 16000 Hz
Length of the rAV audio audio: 2.17 seconds
Number of samples: 34701
Sampling rate: 16000 Hz
Length of the rAV audio audio: 1.87 seconds
Number of samples: 29896
Sampling rate: 16000 Hz
Length of the rAV audio audio: 2.04 seconds
Number of samples: 32566
Sampling rate: 16000 Hz


### About Datasets
**TESS**: Sampling rate of 24 414 Hz, audio files are between 1-2 seconds long

**SAVEE**: Sampling rate of 48000 Hz, audio files are between 3-4 seconds long

**RAVDESS**: Sampling rate of 48000 Hz, audio files are also between 3-4 seconds long

**CREMA**: Sampling rate of 16000 Hz Hz, audio files 1-2 secs long